# Process Performance and Reliability Data Sets



In [3]:
import sys
import pandas as pd 
sys.path.append('..')
from Model.Fleet import Fleet

avg_path = '../southernoperations/AIID/AVG'
compressed_path = '../southernoperations/AIID/CMP'
yaw_path = '../southernoperations/AIID/YAW'
oem_powercurves_path = '../assets/data/power_curves/all_power_curves.csv'

fleet = Fleet(avg_dir=avg_path,cmp_dir=compressed_path, yaw_dir=yaw_path, oem_powercurves_path=oem_powercurves_path, single_plant="WHE")

powercurves = fleet.get_powercurves()
powercurve_distributions = fleet.get_powercurve_distributions()
daily_severity_scores = fleet.daily_severity_scores
daily_severity_scores.columns = [x.replace("-EFFICIENCY","-SEVERITY") for x in daily_severity_scores.columns]
daily_efficiencies =fleet.get_daily_efficiency()
daily_lost_energy = fleet.get_daily_lost_energy()
daily_lost_revenue = fleet.get_daily_lost_revenue()
daily_means = fleet.get_daily_mean()
daily_severity_scores = pd.concat([daily_severity_scores,daily_efficiencies,daily_lost_energy,daily_lost_revenue,daily_means], axis=1)

daily_severity_scores.to_csv('../southernoperations/AIID/fleet_daily_severity_scores_stage.csv')
powercurves.to_csv('../southernoperations/AIID/power_curve.csv')
powercurve_distributions.to_csv('../southernoperations/AIID/power_curve_counts.csv')

Fleet 125: Processing WHE...
Fleet 391: Calculating Power Curves for WHE
[WindFarm 496] processing severity WHE, Xfmr_Winding1_Temp...
[WindFarm 496] processing severity WHE, Xfrm_Aux_Temp...
[WindFarm 496] processing severity WHE, blade_pitch_a...
[WindFarm 496] processing severity WHE, blade_pitch_b...
[WindFarm 496] processing severity WHE, blade_pitch_c...
[WindFarm 496] processing severity WHE, Ctrl_Gnd_Temp...
[WindFarm 496] processing severity WHE, Ctrl_Hub_Temp...
[WindFarm 496] processing severity WHE, Ctrl_Top_Temp...
[WindFarm 496] processing severity WHE, Ctrl_VCP_Temp...
[WindFarm 496] processing severity WHE, Nacl_AD_ADj_Wind_Speed...
[WindFarm 496] processing severity WHE, expected_power...
[WindFarm 496] processing severity WHE, Gbx_CoolingWater_Temp...
[WindFarm 496] processing severity WHE, Gbx_Oil_Temp...
[WindFarm 496] processing severity WHE, Main_Brg_Temp...
[WindFarm 496] processing severity WHE, Gen_Brg_DE_Temp...
[WindFarm 496] processing severity WHE, Gen_Brg_

# Process Fleet Output
This finalizes the processed data for use in the UI performance and reliability area 

In [ ]:
import pandas as pd 

treemap_data = pd.read_csv('../southernoperations/AIID/fleet_daily_severity_scores_stage.csv', index_col=[0], parse_dates = [0])
try:
    treemap_data = treemap_data[~treemap_data.index.to_series().str.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}')]
except:
    pass

treemap_data.index =pd.to_datetime(treemap_data.index)

type_func = lambda x: '-'.join(x.split('-')[2:])
col_suffixes = sorted(list(set([type_func(x) for x in treemap_data.columns])))

temp_treemap_data= treemap_data[[x for x in treemap_data.columns if not any(y in x for y in ['KW','SIMPLE-EFFICIENCY'])]]

temp_treemap_data.to_csv('../southernoperations/AIID/treemap_data.csv')

efficiency_treemap = treemap_data[[x for x in treemap_data.columns if any(y in type_func(x) or y.endswith('_mean') for y in ['KW',
                                                                                           'EFFICIENCY',
                                                                                           'SEVERITY',
                                                                                           'LOST-ENERGY',
                                                                                           'LOST-REVENUE'])]]
efficiency_treemap.to_csv('../southernoperations/AIID/treemap_data_simple_efficiency.csv')

yaw_error_treemap_data = treemap_data[[x for x in treemap_data.columns if any(y in x for y in ['YAW-ERROR'])]]
yaw_error_treemap_data.to_csv('../southernoperations/AIID/radial_yaw_error.csv')

print("Data output to southernoperations/AIID")

# Process Fault Analysis Data Sets

Processes raw 10 minute average and compressed fault code data into data ready to be consumed  by the UI in the fault analysis area

In [ ]:
import sys
import os
import datetime
import pandas as pd

sys.path.append('..')
from Model.Fault import FaultAnalysis

projects = [
            # 'BR2', 
            # 'BTH', 
            # 'CFW', 
            # 'DHW', 
            # 'GNT', 
            # 'GNP', 
            # 'GSW', 
            # 'KAY', 
            'PDK', 
            #  'RDG', 
            #  'SFK', 
            #  'SKO', 
            #  'TBF', 
            #  'WAK', 
            #  'WHE'
            ]

fault_analysis_objects = {}

for project in projects:
    
    fault_data_path = f"../southernoperations/AIID/CMP/{project}.csv"
    
    avg_data_path = f"../southernoperations/AIID/AVG/{project}_concat.csv"
    
    print('Processing', project)
    
    cmp_data = pd.read_csv(fault_data_path, low_memory=False)
    avg_data = pd.read_csv(avg_data_path, index_col=[0], parse_dates=[0])

    downtime = FaultAnalysis(project=project, cmp_data=cmp_data, avg_data=avg_data)

    downtime_data = downtime.reshaped_data
    
    #put each object in memory for use after run 
    fault_analysis_objects[project] = downtime
    output_data = downtime_data.loc[downtime_data['StartDateTime'] > '8/1/2023'].dropna()
    output_data.to_csv(f'../southernoperations/AIID/Processed Faults/{project}_downtime_lost_energy.csv', index=False)

    # daily file output
    daily_data = downtime.daily_downtime()
    daily_data['FaultCount'] = downtime.daily_fault_count()["FaultCount"]
    daily_data['LostEnergy'] = downtime.daily_lost_energy()['LostEnergy']
    daily_data['LostRevenue'] = downtime.daily_lost_revenue()['LostRevenue']

    daily_data.to_csv(f'../southernoperations/AIID/Processed Faults/{project}-daily-fault.csv', index=False)

    print('Fault data transformed and output to "southernoperations/AIID/Processed Faults"')


# Concatenate Fault Metric Files 

In [3]:
import pandas as pd
import os

# Define the directory path
directory_path = '../southernoperations/AIID/Processed Faults/'
search_strings = ['daily-fault.csv','downtime_lost_energy.csv']
output_file_names = ['../southernoperations/AIID/daily_turbine_fault.csv','../southernoperations/AIID/downtime_lost_energy.csv']

for i, search_str in enumerate(search_strings):
    output_file_name = output_file_names[i]
    
    # Get all files in the directory
    all_files = os.listdir(directory_path)

    # Filter files that contain the words 'downtime_lost_energy.csv'
    filtered_files = [f for f in all_files if search_str in  f]

    # Initialize an empty list to store DataFrames
    dfs = []

    # Read and store each file in the list
    for file in filtered_files:
        file_path = os.path.join(directory_path, file)
        dfs.append(pd.read_csv(file_path))

    # Concatenate all DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    # Save the concatenated DataFrame to a new file
    output_file = output_file_name
    combined_df =combined_df.dropna().reset_index()
    if "Unnamed: 0" in combined_df.columns:
        combined_df = combined_df.drop("Unnamed: 0", axis = 1)
    combined_df.to_csv(output_file, index=False)

    print(f"Concatenated {len(filtered_files)} files and saved to {output_file}")



Concatenated 14 files and saved to ../southernoperations/AIID/daily_turbine_fault.csv
Concatenated 15 files and saved to ../southernoperations/AIID/downtime_lost_energy.csv
